![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
### **Programming Quantum Computers** by O'Reilly Media -  [**book info**](http://shop.oreilly.com/product/0636920167433.do)  - [**all code samples**](https://oreilly-qc.github.io)

### **Code samples for Chapter 5**
These code samples were written by Mariia Mykhailova.

#### Example 5-1: Integer increment-by-one operation

In [ ]:
// Example 5-1: Integer increment-by-one operation (circuit implementation)

open Microsoft.Quantum.Diagnostics;

// Circuit implementation of modular increment operation
operation CircuitIncrement (register : Qubit[]) : Unit is Adj {
    for (i in Length(register) - 1 .. -1 .. 0) {
        Controlled X(register[0 .. i - 1], register[i]);
    }
}

operation CircuitIncrementAndDecrement () : Unit {
    // allocate the qubit register
    using (register = Qubit[4]) {
        // initialize the inputs
        X(register[0]);
        H(register[2]);
        T(register[2]);
        
        DumpMachine(());
        // note that at this point non-zero amplitudes are at the basis states |1❭ and |5❭

        Message("Increment (circuit implementation)");
        Increment(register);
        DumpMachine(());
        // note that after incrementing the same amplitudes shifted to the basis states |5❭ and |6❭
        
        // decrement is an adjoint of increment, so we don't need to implement it separately
        Message("Decrement (curcuit implementation)");
        Adjoint Increment(register);
        DumpMachine(());
        
        // make sure the qubits are back to the |0❭ state
        ResetAll(register);
    }
}

In [ ]:
%simulate CircuitIncrementAndDecrement

In [ ]:
// Example 5-1: Integer increment-by-one operation (using library operation)

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation LibraryIncrementAndDecrement () : Unit {
    // allocate the qubit register
    using (register = Qubit[4]) {
        // initialize the inputs
        X(register[0]);
        H(register[2]);
        T(register[2]);
        
        DumpMachine(());
        // note that at this point non-zero amplitudes are at the basis states |1❭ and |5❭

        // Increment the register using library implementation of the increment from Microsoft.Quantum.Arithmetic
        Message("Increment (library implementation)");
        IncrementByInteger(1, LittleEndian(register));
        DumpMachine(());

        Message("Decrement (library implementation)");
        IncrementByInteger(-1, LittleEndian(register));
        DumpMachine(());

        // make sure the qubits are back to the |0❭ state
        ResetAll(register);
    }
}

In [ ]:
%simulate LibraryIncrementAndDecrement

#### Example 5-2: Adding two quantum integers

In [ ]:
%package Microsoft.Quantum.Numerics

In [ ]:
// Example 5-2: Adding two quantum integers

open Microsoft.Quantum.Diagnostics;

// Open namespace which defines arithmetic operations
open Microsoft.Quantum.Arithmetic;

operation QuantumAddition () : Unit {
    using ((a, b) = (Qubit[4], Qubit[2])) {
        // Initialize the inputs
        X(a[0]);
        H(a[2]);
        T(a[2]);
        Message("Register a");
        DumpRegister((), a);
        // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

        X(b[0]);
        H(b[1]);
        T(b[1]);
        Message("Register b");
        DumpRegister((), b);
        // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

        // The library operation AddI will add the first parameter to the second one
        AddI(LittleEndian(b), LittleEndian(a));
        
        // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
        Message("Stage of the system after computing a += b");
        DumpMachine(());
        // The output of this operation is a bit tricky to decipher, since it represents both registers a and b.
        // Register a is the lowest 4 bits of the overall state, and register b is the highest 2 bits.
        // Thus, the non-zero amplitude 0.5 of the basis state |18❭ corresponds to the little-endian bit mask 010010,
        // which can be split up into registers a and b as 0100 10, 
        // which in turn is converted to decimal numbers 2 and 1:
        // the first number is the basis state of register a, storing the sum of basis states |1❭ and |1❭,
        // and the second number is the input state of register b (|1❭).
        // The amplitude of this state is a product of the amplitudes of the basis states |1❭ and |1❭ that were added, 1/sqrt(2) * 1/sqrt(2).
        // The rest of the states can be traced in a similar way.
        
        // Make sure the qubits are back to the |0❭ state
        ResetAll(a + b);
    }
}

In [ ]:
%simulate QuantumAddition

#### Example 5-3: Some interesting arithmetic

In [ ]:
// Example 5-3: Some interesting arithmetic

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation AddBSquared () : Unit {
    using ((a, b) = (Qubit[4], Qubit[2])) {
        // Initialize the inputs
        X(a[0]);
        H(a[2]);
        T(a[2]);
        Message("Register a");
        DumpRegister((), a);
        // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

        X(b[0]);
        H(b[1]);
        T(b[1]);
        Message("Register b");
        DumpRegister((), b);
        // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

        // Allocate extra qubits to hold the value of b² temporarily;
        // note that the register has to have twice the number of qubits in register b.
        using (anc = Qubit[4]) {
            // Compute anc = b * b
            SquareI(LittleEndian(b), LittleEndian(anc));
            
            // Compute a += anc
            AddI(LittleEndian(anc), LittleEndian(a));
            
            // Uncompute the extra qubits, so that they can be released
            Adjoint SquareI(LittleEndian(b), LittleEndian(anc));
        }

        // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
        Message("Stage of the system after computing a += b * b");
        DumpMachine(());
        // The output of this operation can be deciphered in a same way as the previous one:
        // basis states |18❭ and |22❭ remain unchanged, since they correspond to the number 1 written in register b;
        // basis state |58❭ corresponds to little-endian binary 010111, which represents b = 3 and a + b * b = 1 + 3² = 10,
        // and basis state |62❭ is binary 011111, which is b = 3 and a + b * b = 5 + 3² = 14. 
        
        // Make sure the qubits are back to the |0❭ state
        ResetAll(a + b);
    }
}

In [ ]:
%simulate AddBSquared

#### Example 5-4: Quantum-conditional execution

In [ ]:
// Example 5-4: Quantum-conditional execution

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation QuantumConditionalExecution () : Unit {
    using ((a, b) = (Qubit[3], Qubit[3])) {
        // Initialize the inputs
        X(a[0]);
        H(a[2]);
        Message("Register a");
        DumpRegister((), a);
        // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

        X(b[0]);
        H(b[1]);
        T(b[1]);
        Message("Register b");
        DumpRegister((), b);
        // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

        // a -= 3
        IncrementByInteger(-3, LittleEndian(a));
        
        // if (a < 0) then b++;
        (Controlled IncrementByInteger)([a[2]], (1, LittleEndian(b)));

        // a += 3
        IncrementByInteger(3, LittleEndian(a));
        
        // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
        Message("Stage of the system after the computation");
        DumpMachine(());
        
        // Make sure the qubits are back to the |0❭ state
        ResetAll(a + b);
    }
}

In [ ]:
%simulate QuantumConditionalExecution

#### Example 5-5: Phase-encoding the result

In [ ]:
// Example 5-5: Phase-encoding the result

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation PhaseEncoding () : Unit {
    using ((a, b) = (Qubit[3], Qubit[2])) {
        // Initialize the inputs
        H(a[0]);
        H(a[1]);
        Message("Register a");
        DumpRegister((), a);
        // At this point non-zero amplitudes of register a are at the basis states |0❭, |1❭, |2❭, and |3❭

        X(b[0]);
        H(b[1]);
        Message("Register b");
        DumpRegister((), b);
        // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

        Message("Complete state of the system");
        DumpMachine(());

        // a -= 3
        IncrementByInteger(-3, LittleEndian(a));
        
        // if (a < 0) and b == 1 then flip the phase
        (ControlledOnInt(1, Z))(b, a[2]);

        // a += 3
        IncrementByInteger(3, LittleEndian(a));
        
        // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
        Message("Stage of the system after the computation");
        DumpMachine(());
        
        // Make sure the qubits are back to the |0❭ state
        ResetAll(a + b);
    }
}

In [ ]:
%simulate PhaseEncoding

#### Example 5-6: Logic using CNOT gates

In [ ]:
// Example 5-6: Logic using CNOT gates

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation CNOTLogic () : Unit {
    Message("c = ~c");
    using (c = Qubit()) {
        Message($"Input: c = {M(c)}");
        X(c);
        Message($"Output: c = {M(c)}");
        ResetAll([c]);
    }
    
    Message("");
    Message("if (b) then c = ~c");
    using ((b, c) = (Qubit(), Qubit())) {
        // initialize input
        X(b);
        Message($"Input: b = {M(b)}, c = {M(c)}");
        
        CNOT(b, c);
        Message($"Output: b = {M(b)}, c = {M(c)}");

        ResetAll([b, c]);
    }
    
    Message("");
    Message("if (a and b) then c = ~c");
    using ((a, b, c) = (Qubit(), Qubit(), Qubit())) {
        // initialize input
        X(a);
        X(b);
        Message($"Input: a = {M(a)}, b = {M(b)}, c = {M(c)}");
        
        (Controlled X)([a, b], c);
        Message($"Output: a = {M(a)}, b = {M(b)}, c = {M(c)}");

        ResetAll([a, b, c]);
    }
}

In [ ]:
%simulate CNOTLogic